## Library Import

In [8]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm

## Data Load

In [9]:
police_border = gpd.read_file('./data/1.5개시도_경찰서_관할경계.geojson')
police_border.head(3)

,NAME,PNAME,geometry
0,세종경찰서,충남청,"MULTIPOLYGON (((127.17202 36.73106, 127.17202 ..."
1,진주경찰서,경남청,"MULTIPOLYGON (((128.26697 35.12927, 128.26697 ..."
2,창원서부경찰서,경남청,"MULTIPOLYGON (((128.63363 35.22152, 128.63357 ..."


In [10]:
emergency_bell = gpd.read_file('./data/22.5개시도_비상벨현황.geojson')
emergency_bell.head(3)

,bell_us,bell_set_fac,bell_loc,bell_set_yr,geometry
0,기타,화장실,오가낭뜰 근린공원(간이),2017,MULTIPOINT (127.24880 36.51230)
1,기타,화장실,어진동음악분수,2017,MULTIPOINT (127.25660 36.50190)
2,기타,화장실,제천(간이),2017,MULTIPOINT (127.25660 36.50190)


In [11]:
pub_con = pd.read_csv('./data/21.5개시도_유흥업소_단란주점현황.csv')
pub_con.head(3)

,license_date,open_gbn,type,surroundings,lon,lat
0,19951012,영업/정상,단란주점,NaN,127.300127,36.601953
1,20181127,영업/정상,단란주점,NaN,127.301023,36.601199
2,20021121,영업/정상,단란주점,유흥업소밀집지역,127.300940,36.601856


# --------------------------------------------------

## ※ 목표 : Q1 ~ Q5 예측 시 X값으로 유흥업소와 비상벨 개수를 활용 (0.2 과정과 동일)
- 각 lon, lat 데이터에 해당하는 관할서를 매칭시킨 결과를 새 컬럼에 표기

In [12]:
from tqdm import tqdm
tqdm.pandas()

### 1. 비상벨 현황(emergency_bell)

In [9]:
emergency_bell.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 18334 entries, 0 to 18333
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   bell_us       18334 non-null  object  
 1   bell_set_fac  18334 non-null  object  
 2   bell_loc      18333 non-null  object  
 3   bell_set_yr   18334 non-null  int64   
 4   geometry      18334 non-null  geometry
 5   jur_stn       18334 non-null  object  
dtypes: geometry(1), int64(1), object(4)
memory usage: 859.5+ KB


In [13]:
def detect_police_name(geometry) :
    for i in range(len(police_border['NAME'].unique())) :    # 보안등 위치의 geometry 와 관할서의 geometry를 비교하여
        if geometry.within(police_border.iloc[i].geometry) :             #  매칭된 관할서명을 return하는 함수
            return police_border.iloc[i].NAME            

In [7]:
emergency_bell['jur_stn'] = '' # 함수 적용
emergency_bell['jur_stn'] = emergency_bell.progress_apply(lambda x : detect_police_name(x['geometry']) , axis=1 )

100%|██████████| 18334/18334 [16:26<00:00, 18.59it/s]


In [16]:
emergency_bell.head()

서울중랑경찰서     1780
서울광진경찰서     1519
서울동작경찰서     1107
서울영등포경찰서    1102
서울용산경찰서     1024
Name: jur_stn, dtype: int64

In [17]:
# 추후 활용을 위해 csv file로 저장
emergency_bell.to_csv('emergency_bell.csv')

### 2. 유흥업소현황(pub_con)

In [14]:
pub_con.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7119 entries, 0 to 7118
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   license_date  7119 non-null   int64  
 1   open_gbn      7119 non-null   object 
 2   type          7118 non-null   object 
 3   surroundings  3525 non-null   object 
 4   lon           7119 non-null   float64
 5   lat           7119 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 333.8+ KB


In [15]:
# csv to geopandas
# lon, lat data를 geometry로 변경
pub_con['lat'] = pub_con['lat'].astype(float)
pub_con['lon'] = pub_con['lon'].astype(float)
pub_con['geometry'] = pub_con.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
pub_con = gpd.GeoDataFrame(pub_con, geometry='geometry')
pub_con.crs = {'init':'epsg:4326'}
pub_con = pub_con.to_crs({'init':'epsg:4326'}) # 좌표계 epsg : 4326

/opt/app-root/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [16]:
pub_con['jur_stn'] = '' # 함수 적용
pub_con['jur_stn'] = pub_con.progress_apply(lambda x : detect_police_name(x['geometry']) , axis=1 )

100%|██████████| 7119/7119 [01:50<00:00, 64.32it/s] 


In [24]:
pub_con.head()

,license_date,open_gbn,type,surroundings,lon,lat,geometry,jur_stn,year,month,date
0,19951012,영업/정상,단란주점,NaN,127.300127,36.601953,POINT (127.30013 36.60195),세종경찰서,1995,10,1995(하반기)
1,20181127,영업/정상,단란주점,NaN,127.301023,36.601199,POINT (127.30102 36.60120),세종경찰서,2018,11,2018(하반기)
2,20021121,영업/정상,단란주점,유흥업소밀집지역,127.300940,36.601856,POINT (127.30094 36.60186),세종경찰서,2002,11,2002(하반기)
3,20020409,영업/정상,단란주점,유흥업소밀집지역,127.299671,36.602782,POINT (127.29967 36.60278),세종경찰서,2002,4,2002(상반기)
4,19990804,영업/정상,단란주점,학교정화(상대),127.205025,36.678746,POINT (127.20503 36.67875),세종경찰서,1999,8,1999(하반기)


In [25]:
# 2020년 이후 허가받은 업소 제외
pub_con= pub_con[pub_con['year']<2021]

In [26]:
# 17년 이전 허가받은 업소는 다 17년도로 지정(누적 count하기 위함)
pub_con['set_year'] = pub_con['year'].apply(lambda x : 2017 if x<2017 else x)
pub_con.head(3)

,license_date,open_gbn,type,surroundings,lon,lat,geometry,jur_stn,year,month,date,set_year
0,19951012,영업/정상,단란주점,NaN,127.300127,36.601953,POINT (127.30013 36.60195),세종경찰서,1995,10,1995(하반기),2017
1,20181127,영업/정상,단란주점,NaN,127.301023,36.601199,POINT (127.30102 36.60120),세종경찰서,2018,11,2018(하반기),2018
2,20021121,영업/정상,단란주점,유흥업소밀집지역,127.300940,36.601856,POINT (127.30094 36.60186),세종경찰서,2002,11,2002(하반기),2017


In [38]:
# license data에서 인덱싱하여 year, month 생성
pub_con['year'] = pub_con['license_date'].apply(lambda x :int(str(x)[:4]))
pub_con['month'] = pub_con['license_date'].apply(lambda x :int(str(x)[4:6]))

In [40]:
# 관할서의 연도별 유흥업소 count
pub_cnt = pub_con.groupby(['jur_stn','set_year']).count().unstack(fill_value=0).stack()[['type']]
pub_cnt.reset_index(inplace=True)
pub_cnt.head(3)

,jur_stn,set_year,type
0,마산동부경찰서,2017,329
1,마산동부경찰서,2018,1
2,마산동부경찰서,2019,2


In [41]:
# 2017년의 count를 이후 년도에 누적해주어야 함. 
# 2017년 이후의 값 = 2017 값 + a을 해주는 작업
for j in pub_cnt['jur_stn'].unique() :
    type_2017 = pub_cnt[(pub_cnt['jur_stn']==j) & (pub_cnt['set_year'] == 2017)]['type'].values[0]
    pub_cnt['type'] = pub_cnt.apply(lambda x : x['type'] + type_2017 if x['jur_stn']==j and x['set_year'] != 2017 else x['type'], axis=1)
pub_cnt.head()

,jur_stn,set_year,type
0,마산동부경찰서,2017,329
1,마산동부경찰서,2018,330
2,마산동부경찰서,2019,331
3,마산동부경찰서,2020,331
4,마산중부경찰서,2017,466


In [44]:
pub_cnt = pub_cnt.rename(columns={'set_year' :'year', 'type' : 'pub_cnt'})
pub_cnt.head()

,jur_stn,year,pub_cnt
0,마산동부경찰서,2017,329
1,마산동부경찰서,2018,330
2,마산동부경찰서,2019,331
3,마산동부경찰서,2020,331
4,마산중부경찰서,2017,466


In [45]:
pub_cnt.to_csv('pub_cnt.csv')

### 상, 하반기에 나눠 담기
- year 기준으로 cnt 되어있으므로 상, 하반기에 각 각 담아준다

In [47]:
## 인구종합.csv를 불러와서 merge할수 있도록 year 컬럼을 만들어준다.
total_pop = pd.read_csv('인구정보종합.csv')
jur_date_df = total_pop[['jur_stn','date']]
jur_date_df['year'] = jur_date_df['date'].apply(lambda x : int(x[:4]))
jur_date_df.head(3)

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,jur_stn,date,year
0,서울중부경찰서,2017(상반기),2017
1,서울종로경찰서,2017(상반기),2017
2,서울남대문경찰서,2017(상반기),2017


In [48]:
jur_date_df['year'].value_counts()

2020    82
2019    82
2018    82
2017    82
Name: year, dtype: int64

In [49]:
# 만든 base_df에 관할서, 연도 기준으로 Merge
pub_con_df = pd.merge(jur_date_df, pub_cnt, on=['jur_stn','year'])
pub_con_df.head()

,jur_stn,date,year,pub_cnt
0,서울중부경찰서,2017(상반기),2017,201
1,서울중부경찰서,2017(하반기),2017,201
2,서울종로경찰서,2017(상반기),2017,149
3,서울종로경찰서,2017(하반기),2017,149
4,서울남대문경찰서,2017(상반기),2017,87


In [55]:
pub_con_df['date'].value_counts() # 데이터 개수 확인

2018(하반기)    41
2017(상반기)    41
2017(하반기)    41
2019(상반기)    41
2020(하반기)    41
2019(하반기)    41
2018(상반기)    41
2020(상반기)    41
Name: date, dtype: int64

In [56]:
pub_con_df.head()

,jur_stn,date,year,pub_cnt
0,서울중부경찰서,2017(상반기),2017,201
1,서울중부경찰서,2017(하반기),2017,201
2,서울종로경찰서,2017(상반기),2017,149
3,서울종로경찰서,2017(하반기),2017,149
4,서울남대문경찰서,2017(상반기),2017,87


In [57]:
# 추후 활용을 위해 csv file로 저장
pub_con_df.to_csv('유흥업소현황.csv')